# Hashcode Question Classification

## 1. RNN Classifier

In [87]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# for counting
import collections

import re

import os
print(os.listdir("./input"))
%matplotlib inline 

['hashcode_classification2020_test.csv', 'hashcode_classification2020_sample.csv', '.ipynb_checkpoints', 'hashcode_classification2020_train.csv']


## EDA

In [88]:
train_df = pd.read_csv('./input/hashcode_classification2020_train.csv', encoding='utf-8')
train_df.head(5)

,title,content,label
0,wait()과 sleep()의 차이점은 뭔가요,###발생하는 문제 및 실행환경\nwait()과 sleep()의 차이점은 뭔가요,3
1,$$$ 초보 외부 클래스 멤버 변수 사용 질문합니다.ㅠㅠ,"1.헤더에, 사용할 멤버변수가 담긴 헤더 파일이 Include 되어있습니다.\n예를...",2
2,ORM: Sequelize: 다대다 관계 쿼리,"안녕하세요.\n\n어떻게 다대다 관계 쿼리를 해야하나요? 예를들어, `product...",4
3,$$$에서 숫자가 NaN인지 검사하려면 어떻게해야하죠?,```\nparseFloat('geoff') == NaN;\n\nparseFloat...,4
4,"$$$ 2.7에서 중국어, 특수문자 인코딩 하는 방법이 뭔가요?",```\n>>>dict['name']\n胡安·马塔\n>>>json.dumps(dic...,5


In [89]:
print(train_df.loc[(train_df['content'].isna() == True)])
print(train_df.loc[(train_df['title'].isna() == True)])

                                                  title content  label
2400  [$$$] 재귀함수를 처리하면서 setTimeout으로 딜레이를 주면 읽는 순서가 ...     NaN      4
Empty DataFrame
Columns: [title, content, label]
Index: []


In [90]:
train_df.loc[(train_df['content'].isna() == True)] = ""
train_df.loc[(train_df['content'].isna() == True)]

,title,content,label


* label이 하나만 존재하는지 확인

In [92]:
train_df['label'].hist(bins=train_df['label'].max())

TypeError: '>=' not supported between instances of 'int' and 'str'

## 데이터 전처리

In [63]:
re.sub(r'[$]', r'', "$$$ 초보 외부 클래스 멤버 변수 사용 질문합니다.ㅠ")

' 초보 외부 클래스 멤버 변수 사용 질문합니다.ㅠ'

In [61]:
re.sub(r'\n', r'', "== NaN;\n == Number.NaN;\n 해봤는데 둘다 true로만 나와요.")

'== NaN; == Number.NaN; 해봤는데 둘다 true로만 나와요.'

In [93]:
def cleansing_text(text):
    
    try:
        text = re.sub(r'[#$`\n\t*]', r'', text)
    except:
        pass
    return text

train_df['title'] = train_df['title'].map(lambda text: cleansing_text(text))
train_df['content'] = train_df['content'].map(lambda text: cleansing_text(text))

train_df.head(50)

,title,content,label
0,wait()과 sleep()의 차이점은 뭔가요,발생하는 문제 및 실행환경wait()과 sleep()의 차이점은 뭔가요,3
1,초보 외부 클래스 멤버 변수 사용 질문합니다.ㅠㅠ,"1.헤더에, 사용할 멤버변수가 담긴 헤더 파일이 Include 되어있습니다.예를 들...",2
2,ORM: Sequelize: 다대다 관계 쿼리,"안녕하세요.어떻게 다대다 관계 쿼리를 해야하나요? 예를들어, product, cat...",4
3,에서 숫자가 NaN인지 검사하려면 어떻게해야하죠?,parseFloat('geoff') == NaN;parseFloat('geoff')...,4
4,"2.7에서 중국어, 특수문자 인코딩 하는 방법이 뭔가요?",>>>dict['name']胡安·马塔>>>json.dumps(dict['name']...,5
5,으로 탐색기 시작위치 변경하고싶습니다.,으로 코딩중인데 def explorer(): subprocess.cal...,5
6,웹 개발 하고 있습니다. 실무에서 lombok 사용하나요?,으로 웹 개발 하고 있습니다.회사에서는 사수가 주는 코트만 수정중이라...개인적으로...,3
7,"express에서 특정 단추를 눌렀을 때, mysql 데이터를 수정한 후, 수정된 ...","main.jsapp.post('/count', function(req, res){v...",4
8,텍스트 파일을 읽어와서 내림차순 혹은 오름차순으로 정리하고 다시 파일로 출력,9 김철수 7210 박재용 791 나영희 343 이나연 46이런식으로 되어있는 텍스...,3
9,클래스 변수에 대해 궁금합니다,Class Dog : name = str() trick = []A = D...,5


In [81]:
from __future__ import division
from pprint import pprint
import _pickle as cPickle

from gensim.models import Doc2Vec, Phrases
from gensim.models.doc2vec import LabeledSentence
from gensim.matutils import unitvec
import numpy as np
from numpy import dot
import pandas as pd
from random import shuffle
from collections import Counter

from gensim.parsing.preprocessing import STOPWORDS as stop_words
letters = list('abcdefghijklmnopqrstuvwxyz')
numbers = list('123456789')
stop_words = stop_words.union(set(letters)).union(set(numbers))

import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [96]:
import string
import re
RE_PUNCT = re.compile('([%s])+' % re.escape(string.punctuation), re.UNICODE)

def preprocess(text):
    # Remove all punctuation and make all lowercase 
    return RE_PUNCT.sub(r" ", text).lower().split()

In [100]:
def make_doc(text, title, drop_stopwords=True):
    """Make documents into LabeledSentence objects for doc2vec training"""
       
    docwords = list(filter(lambda word: word not in stop_words,
                      bigram[text]))
    return LabeledSentence(docwords, [title])

# Train bigrammer to detect two-word phrases, e.g., breaking_bad
#bigram = Phrases(map(preprocess, df.rev_text.tolist())) 
bigram = Phrases(map(preprocess, train_df['content'])) 

#DOCS = [make_movie_doc(text, title) for text, title in
#        zip(df.rev_text.tolist(), df.title.tolist())]
DOCS = [make_doc(train_df['content'], train_df['title'] ) for _, row
        in train_df.iterrows()]

2020-02-02 18:04:22,998 : INFO : collecting all words and their counts
2020-02-02 18:04:22,999 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2020-02-02 18:04:23,870 : INFO : collected 231975 word types from a corpus of 309160 words (unigram + bigrams) and 2592 sentences
2020-02-02 18:04:23,871 : INFO : using 231975 counts as vocab in Phrases<0 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>
/Users/DongGyun/anaconda/envs/nlu/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  


In [101]:
model = Doc2Vec(dm=0, dbow_words=1, min_count=4, negative=3,
                hs=0, sample=1e-4, window=10, size=100, workers=8)

model.build_vocab(DOCS)
model.train(DOCS, total_examples=model.corpus_count, epochs=model.iter)

/Users/DongGyun/anaconda/envs/nlu/lib/python3.7/site-packages/gensim/models/doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
2020-02-02 18:07:38,155 : INFO : collecting all words and their counts
2020-02-02 18:07:38,175 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags


TypeError: 'Series' objects are mutable, thus they cannot be hashed

## Multi-layer bi-LSTM with dropout?

* Pytorch의 nn.lstm은 멀티 레이어, 양방향 레이어, dropout을 파라미터로 설정해줄 수 있습니다.
* RNN 계열에서 dropout은 [Recurrent Neural Network Regularization](https://arxiv.org/abs/1409.2329) 논문에서 처음으로 잘 동작하도록 사용했다고 합니다. ([관련 블로그 글](http://sanghyukchun.github.io/89/))
* 중요한 부분은 아래 option2 그림처럼 multi-layer에서 dropout이 사용되며, dropout은 recurrent connection이 아닌 부분에만 적용해야 하는 것이라고 합니다.

* Reference: (https://discuss.pytorch.org/t/lstm-dropout-clarification-of-last-layer/5588)

![multi-layer LSTM with dropout](https://discuss.pytorch.org/uploads/default/original/2X/6/62f94ceee433b693ef73be231f51ae4291e53880.png)

## 2. CNN Classifier

![](http://i.imgur.com/JN72JHW.png)

* CNN 텍스트 분류기 구조 ([Yoon Kim(2014)](http://emnlp2014.org/papers/pdf/EMNLP2014181.pdf))

* CNN을 사용한 텍스트 분류:
 * 문장의 단어들을 window 단위로 convolution 하여 feature를 추출한다.
 
저자의 구현은 아래와 같습니다.

In [64]:
# %load simple-ntc/simple_ntc/cnn.py
import torch
import torch.nn as nn


class CNNClassifier(nn.Module):

    def __init__(self,
                 input_size,
                 word_vec_dim,
                 n_classes,
                 use_batch_norm=False,
                 dropout_p=.5,
                 window_sizes=[3, 4, 5],   # 논문에서 가장 좋은 성능을 보여준 window size & n_filters
                 n_filters=[100, 100, 100]
                 ):
        # CNN 파라미터
        self.input_size = input_size  # vocabulary size
        self.word_vec_dim = word_vec_dim
        self.n_classes = n_classes
        self.use_batch_norm = use_batch_norm
        self.dropout_p = dropout_p
        # window_size means that how many words a pattern covers.
        self.window_sizes = window_sizes
        # n_filters means that how many patterns to cover.
        self.n_filters = n_filters

        super().__init__()

        # CNN 텍스트 분류기 레이어 구조
        self.emb = nn.Embedding(input_size, word_vec_dim)
        # Since number of convolution layers would be vary depend on len(window_sizes),
        # we use 'setattr' and 'getattr' methods to add layers to nn.Module object.
        # -> custom layer을 추가(setattr)하고, 불러와 사용하는 것(getattr)
        for window_size, n_filter in zip(window_sizes, n_filters):
            cnn = nn.Conv2d(in_channels=1,
                            out_channels=n_filter,
                            kernel_size=(window_size, word_vec_dim)
                            )
            setattr(self, 'cnn-%d-%d' % (window_size, n_filter), cnn)

            # 빠른 학습속도와 regularization 효과가 있는 batch-norm
            if use_batch_norm:
                bn = nn.BatchNorm2d(n_filter)
                setattr(self, 'bn-%d-%d' % (window_size, n_filter), bn)
        # Because below layers are just operations, 
        # (it does not have learnable parameters)
        # we just declare once.
        
        if not use_batch_norm:
            self.dropout = nn.Dropout(dropout_p)
        self.relu = nn.ReLU()
        # An input of generator layer is max values from each filter.
        self.generator = nn.Linear(sum(n_filters), n_classes)
        # We use LogSoftmax + NLLLoss instead of Softmax + CrossEntropy
        self.activation = nn.LogSoftmax(dim=-1)

    def forward(self, x):
        # |x| = (batch_size, length)
        x = self.emb(x)
        # zero padding
        # |x| = (batch_size, length, word_vec_dim)
        min_length = max(self.window_sizes)
        if min_length > x.size(1):
            # Because some input does not long enough for maximum length of window size,
            # we add zero tensor for padding.
            pad = x.new(x.size(0), min_length - x.size(1), self.word_vec_dim).zero_()
            # |pad| = (batch_size, min_length - length, word_vec_dim)
            x = torch.cat([x, pad], dim=1)
            # |x| = (batch_size, min_length, word_vec_dim)

        # In ordinary case of vision task, you may have 3 channels on tensor,
        # but in this case, you would have just 1 channel,
        # which is added by 'unsqueeze' method in below:
        x = x.unsqueeze(1)
        # |x| = (batch_size, 1, length, word_vec_dim)

        # 아래는 convolution-pooling을 진행하는 부분, p261참조
        cnn_outs = []
        for window_size, n_filter in zip(self.window_sizes, self.n_filters):
            cnn = getattr(self, 'cnn-%d-%d' % (window_size, n_filter))
            if self.use_batch_norm:
                bn = getattr(self, 'bn-%d-%d' % (window_size, n_filter))
                cnn_out = bn(self.relu(cnn(x)))
            else:
                cnn_out = self.dropout(self.relu(cnn(x))) 
            # |cnn_out| = (batch_size, n_filter, length - window_size + 1, 1)

            # In case of max pooling, we does not know the pooling size,
            # because it depends on the length of the sentence.
            # Therefore, we use instant function using 'nn.functional' package.
            # This is the beauty of PyTorch. :)
            cnn_out = nn.functional.max_pool1d(input=cnn_out.squeeze(-1),
                                               kernel_size=cnn_out.size(-2)
                                               ).squeeze(-1)
            # |cnn_out| = (batch_size, n_filter)
            cnn_outs += [cnn_out]
        # Merge output tensors from each convolution layer.
        cnn_outs = torch.cat(cnn_outs, dim=-1)
        # |cnn_outs| = (batch_size, sum(n_filters))
        y = self.activation(self.generator(cnn_outs))
        # |y| = (batch_size, n_classes)

        return y


## 3. 학습/테스트 데이터

저자의 경우 클리앙 사이트의 글을 크롤링한 후, 게시글의 카테고리 분류를 수행했습니다.

그러나 저자가 데이터를 제공하고 있지 않고, [클리앙 크롤링 코드](https://github.com/kh-kim/clien_crawler)는 제대로 동작하지 않는 것 같아
저는 [AI hub의 한국어 대화 데이터](http://www.aihub.or.kr/content/553)를 사용하겠습니다.

![한국어 대화 데이터 구조](http://www.aihub.or.kr/sites/default/files/%E1%84%83%E1%85%A2%E1%84%92%E1%85%AA%E1%84%83%E1%85%A6%E1%84%8B%E1%85%B5%E1%84%90%E1%85%A5_%E1%84%80%E1%85%AE%E1%84%89%E1%85%A5%E1%86%BC%E1%84%83%E1%85%A9.JPG)

해당 데이터는 AI hub를 통해 직접 다운 받으셔야 합니다.

저는 대화의 카테고리를 분류해보는 실습을 진행하고자 합니다.

학습을 위한 데이터는 제가 공유드린 parsing.py 파일을 사용해 간단하게 생성할 수 있습니다.<br>
그리고 data_preparation.sh 스크립트를 사용해 corpus를 train/valid 셋으로 나눌 수 있습니다.

제가 카테고리 분류를 수행할 데이터는 대략 아래와 같습니다.

In [58]:
%ll data/dialog

total 151128
-rw-r--r--  1 DongGyun  staff   2720816 10  3 16:29 category_corpus.txt
-rw-r--r--  1 DongGyun  staff   2720816 10  3 16:29 category_corpus.txt.shuf
-rw-r--r--  1 DongGyun  staff   2450240 10  3 16:29 category_corpus.txt.shuf.train
-rw-r--r--  1 DongGyun  staff    270576 10  3 16:29 category_corpus.txt.shuf.valid
-rw-------  1 DongGyun  staff  35637997  5  7 16:39 dialog.json
-rw-------  1 DongGyun  staff  23974499  5  7 16:53 dialog.xml
-rw-r--r--  1 DongGyun  staff   2720816 10  1 19:07 dialog_category.data
-rw-r--r--  1 DongGyun  staff         0 10  3 16:29 domain_corpus.txt
-rw-r--r--  1 DongGyun  staff   5885165 10  3 16:29 intent_corpus.txt


In [23]:
!head -10 data/dialog/category_corpus.txt.shuf

펜션/캠핑장	재료는 직접 사 와야 하나요?
상가	건물이 몇 년도에 지어졌어요?
모텔/여관	제일 작은방은 없나요?
pc방	네 화장실 다녀온 후 라면 가지러 오겠습니다
화장품	여기 이 제품 새 제품 있나요?
미용실	남자만 머리할 수 있어요?
옷수선집	영업시간 좀 문의드려요
카페	몇 시에서 몇 시까지 영업하세요?
신발	요 신발은 몇 문이에요?
반찬가게	혹시 현금영수증 돼요?


## 4. Code

저자 코드의 구조를 살펴보면 아래와 같습니다.<br>
pytorch 프로젝트의 일반적인 구성을 파악할 수 있습니다.

In [57]:
%ll simple-ntc

total 72
-rwxr-xr-x  1 DongGyun  staff  8624  9 23 19:52 README.md*
drwxr-xr-x  4 DongGyun  staff   128 10  1 19:22 __pycache__/
-rwxr-xr-x  1 DongGyun  staff  4609  9 23 19:52 classify.py*
-rwxr-xr-x  1 DongGyun  staff  3396  9 23 19:52 data_loader.py*
-rwxr-xr-x  1 DongGyun  staff  1201  9 23 19:52 get_confusion_matrix.py*
drwxr-xr-x  8 DongGyun  staff   256 10  2 22:24 simple_ntc/
-rwxr-xr-x  1 DongGyun  staff  4045  9 23 19:52 train.py*
-rwxr-xr-x  1 DongGyun  staff   725  9 23 19:52 utils.py*


* train.py: 학습 할때 실행하게 될 파이썬 코드입니다. 학습의 파라미터 설정에 대한 정보를 넘겨줄 수 있고, 학습 결과를 어떻게 저장할지 등을 설정합니다.
* classify.py: 학습한 모델을 사용해 분류(inference)를 수행하는 코드입니다.
* data_loader.py: 데이터에서 text와 label은 무엇인지 결정하고, vocal을 생성합니다. 또한 batch 단위로 데이터를 나누어줍니다.
* utils.py, get_confusion_matrix.py: 저자가 모델 평가를 위해 추가한 것으로 보여지는 코드인데, 정확한 사용 방법이나 용도는 모르겠습니다. (https://github.com/kh-kim/simple-ntc#evaluation)

In [16]:
%ll simple-ntc/simple_ntc

total 32
-rwxr-xr-x  1 DongGyun  staff     0  9 23 19:52 __init__.py*
drwxr-xr-x  6 DongGyun  staff   192 10  1 19:20 __pycache__/
-rwxr-xr-x  1 DongGyun  staff  4212  9 23 19:52 cnn.py*
-rwxr-xr-x  1 DongGyun  staff  1449  9 23 19:52 rnn.py*
-rwxr-xr-x  1 DongGyun  staff  3771  9 23 19:52 trainer.py*


* rnn.py & cnn.py: 앞서 설명드렸던 RNN, CNN 모델 파일입니다.
* trainer.py: 저자는 pytorch 모델 학습의 코드를 더 간략하게 작성하기 위하여 ignite를 사용했습니다. 이 파일은 그 ignite 사용에 대한 부분이 포함되어 있습니다. (https://pytorch.org/ignite/#ignite-your-networks)

# 5. Training

In [48]:
%ll data/dialog

total 151128
-rw-r--r--  1 DongGyun  staff   2720816 10  3 16:29 category_corpus.txt
-rw-r--r--  1 DongGyun  staff   2720816 10  3 16:29 category_corpus.txt.shuf
-rw-r--r--  1 DongGyun  staff   2450240 10  3 16:29 category_corpus.txt.shuf.train
-rw-r--r--  1 DongGyun  staff    270576 10  3 16:29 category_corpus.txt.shuf.valid
-rw-------  1 DongGyun  staff  35637997  5  7 16:39 dialog.json
-rw-------  1 DongGyun  staff  23974499  5  7 16:53 dialog.xml
-rw-r--r--  1 DongGyun  staff   2720816 10  1 19:07 dialog_category.data
-rw-r--r--  1 DongGyun  staff         0 10  3 16:29 domain_corpus.txt
-rw-r--r--  1 DongGyun  staff   5885165 10  3 16:29 intent_corpus.txt


\# 콘솔에서 아래 명령어를 통해 학습을 수행합니다. (CPU 사용)

\# RNN <br>
python simple-ntc/train.py --model_fn ./models/rnn_model.pth --train ./data/dialog/category_corpus.txt.shuf.train --valid ./data/dialog/category_corpus.txt.shuf.valid --rnn --gpu_id -1

\# CNN <br>
python simple-ntc/train.py --model_fn ./models/cnn_model.pth --train ./data/dialog/category_corpus.txt.shuf.train --valid ./data/dialog/category_corpus.txt.shuf.valid --cnn --gpu_id -1

\# RNN&CNN ensemble <br>
python simple-ntc/train.py --model_fn ./models/ensemble_model.pth --train ./data/dialog/category_corpus.txt.shuf.train --valid ./data/dialog/category_corpus.txt.shuf.valid --rnn -cnn --gpu_id -1

# 6. Inference

In [65]:
!head -n 10 ./data/dialog/category_corpus.txt.shuf.valid | awk -F'\t' '{ print $2 }' | python simple-ntc/classify.py --model ./models/rnn_model.pth --gpu_id -1 --top_k 1

펜션/캠핑장	재료는 직접 사 와야 하나요?
태권도/합기도학원	건물이 몇 년도에 지어졌어요?
화장품	제일 작은방은 없나요?
일반홀서빙음식점	네 화장실 다녀온 후 라면 가지러 오겠습니다
신발	여기 이 제품 새 제품 있나요?
미용실	남자만 머리할 수 있어요?
토지	영업시간 좀 문의드려요
배달음식점	몇 시에서 몇 시까지 영업하세요?
신발	요 신발은 몇 문이에요?
일반홀서빙음식점	혹시 현금영수증 돼요?


In [66]:
!head -n 10 ./data/dialog/category_corpus.txt.shuf.valid | awk -F'\t' '{ print $2 }' | python simple-ntc/classify.py --model ./models/cnn_model.pth --gpu_id -1 --top_k 1

미용실	재료는 직접 사 와야 하나요?
주택	건물이 몇 년도에 지어졌어요?
카페	제일 작은방은 없나요?
일반홀서빙음식점	네 화장실 다녀온 후 라면 가지러 오겠습니다
가방	여기 이 제품 새 제품 있나요?
미용실	남자만 머리할 수 있어요?
주택	영업시간 좀 문의드려요
일반홀서빙음식점	몇 시에서 몇 시까지 영업하세요?
신발	요 신발은 몇 문이에요?
일반홀서빙음식점	혹시 현금영수증 돼요?


In [61]:
!head -n 10 ./data/dialog/category_corpus.txt.shuf.valid | awk -F'\t' '{ print $2 }' | python simple-ntc/classify.py --model ./models/ensemble_model.pth --gpu_id -1 --top_k 3

펜션/캠핑장 미용실 세탁소	재료는 직접 사 와야 하나요?
주택 상가 의류	건물이 몇 년도에 지어졌어요?
약국 의류 가방	제일 작은방은 없나요?
pc방 일반홀서빙음식점 당구장	네 화장실 다녀온 후 라면 가지러 오겠습니다
화장품 가방 신발	여기 이 제품 새 제품 있나요?
미용실 카페 화장품	남자만 머리할 수 있어요?
차량등록 토지 주택	영업시간 좀 문의드려요
옷수선집 일반홀서빙음식점 세탁소	몇 시에서 몇 시까지 영업하세요?
신발 의류 가방	요 신발은 몇 문이에요?
일반홀서빙음식점 의류 배달음식점	혹시 현금영수증 돼요?
